COMP 215 - LAB 2 Records (NEO)
----------------
#### Name:
#### Date:

This lab exercise is mostly a review of strings, tuples, lists, dictionaries, and functions.

**Building on new concepts from lab 1**:
  * `datetime.date` objects represent a calendar date
  * *list comprehension* provides a compact way to represent map and filter algorithms

**New Python Concepts**:
  * *f-string* simplifies string formatting operations

As usual, the first code cell simply imports all the modules we'll be using...

In [5]:
import json, requests
from datetime import date, datetime
from pprint import pprint    # Pretty Print - built-in python function to nicely format data structures

We'll answer some questions about [Near Earth Objects](https://cneos.jpl.nasa.gov/)
> using NASA's API:  [https://api.nasa.gov/](https://api.nasa.gov/#NeoWS)

You should register for your own API key, (but may use the DEMO_KEY to get started).

First, a short reminder about python dates and [f-strings](https://realpython.com/python-f-strings/)...

In [20]:
today = date.today()   # get a date object representing today's date
print(today, type(today))
# f"..."  is a format string - notice how the variable `today` is formatted into the string
print(f'Today is: {today}')

2026-01-12 <class 'datetime.date'>
Today is: 2026-01-12


TURN IT INTO A LIST COMPREHENSION

### Make a query

Let's get some data from the NEO database...
Here's a query that gets the observation "feed" for today.
(Note: I hard-coded the date below to lock down the data for the lab - ideally used `today()` so the notebook is always up-to-date.)

In [41]:
NASA_NEO_FEED_API_URL = "https://api.nasa.gov/neo/rest/v1/feed"
API_KEY = 'LbMciRqaLzeWBjQQLYDSAfuQOzKBTbxU4prcuHoe'  # substitute your API key here

start_date = '2026-12-01'  #  Future enhancement:  str(date.today())   # Today's date as a string!

feed_params = dict(
    start_date=start_date,
    end_date=start_date,
    api_key=API_KEY,
)

response = requests.request("GET", NASA_NEO_FEED_API_URL, params=feed_params)
response.raise_for_status()  # raise an error if request is not successful

feed_data = json.loads(response.text)  # recall json.loads from lab 1

# TIP: print(data) to see the whole data structure returned, here we grab just the list of NEO's:
n_results = feed_data['element_count']
neos = feed_data['near_earth_objects'][start_date]
print(f'{n_results} Near Earth Objects found for {start_date}')
pprint(neos[:2])

7 Near Earth Objects found for 2026-12-01
[{'absolute_magnitude_h': 18.36,
  'close_approach_data': [{'close_approach_date': '2026-12-01',
                           'close_approach_date_full': '2026-Dec-01 05:33',
                           'epoch_date_close_approach': 1796103180000,
                           'miss_distance': {'astronomical': '0.4303139264',
                                             'kilometers': '64374046.820776768',
                                             'lunar': '167.3921173696',
                                             'miles': '40000177.8615366784'},
                           'orbiting_body': 'Earth',
                           'relative_velocity': {'kilometers_per_hour': '69477.7386852814',
                                                 'kilometers_per_second': '19.299371857',
                                                 'miles_per_hour': '43170.7649069739'}}],
  'estimated_diameter': {'feet': {'estimated_diameter_max': 4149.7795866675,
    

Next we extract just the potentially hazerdous asteroids, using a Comp115-style list accumulator *loop*:

In [42]:
hazards =  []
for item in neos:
  if item['is_potentially_hazardous_asteroid'] is True:
    hazards.append(item)
print(f'{len(hazards)} potentially hazardous asteroids identified on {start_date}.')

2 potentially hazardous asteroids identified on 2026-12-01.


## Exercise 1

In the code cell below, **re-write the accumulator loop above** as a [list comprehension](https://realpython.com/lessons/list-comprehensions-overview/) that implements a ["filter"](https://youtu.be/hUes6y2b--0)
Notice how this provides a concise way to "filter" items of interest from a larger data set.

In [43]:
# Ex. 1 your code here
hazards_comprehension = [neo for neo in neos if neo['is_potentially_hazardous_asteroid'] is True]
(f'{len(hazards_comprehension)} potentially hazardous asteroids identified on {start_date}')

'2 potentially hazardous asteroids identified on 2026-12-01'

## Fetch Complete Data for One Asteroid

Notice that the record for each `neo` is a dictionary with `id` field that uniquely identifies this record in the database.

We can use this `id` to fetch complete orbital and close approach data for the NEO.

For example, this query fetches the complete data set for the first hazardous asteroid...


In [52]:
NASA_NEO_API_URL = "https://api.nasa.gov/neo/rest/v1/neo/{id}"
neo_params = dict(
    api_key=API_KEY,
)
hazard_id = hazards[0]['id']

response = requests.request("GET", NASA_NEO_API_URL.format(id=hazard_id), params=neo_params)
response.raise_for_status()  # raise an error if request is not successful

neo_data = json.loads(response.text)

print(f'NEO {neo_data["name"]} with {len(neo_data["close_approach_data"])} approach records.  {"Hazardous." if neo_data["is_potentially_hazardous_asteroid"] else ""}')
print(f'Close approach records:')
close_approach_data = neo_data['close_approach_data']
pprint(close_approach_data[:2])

NEO 480883 (2001 YE4) with 212 approach records.  Hazardous.
Close approach records:
[{'close_approach_date': '1903-10-11',
  'close_approach_date_full': '1903-Oct-11 18:27',
  'epoch_date_close_approach': -2089863180000,
  'miss_distance': {'astronomical': '0.137647987',
                    'kilometers': '20591845.66498769',
                    'lunar': '53.545066943',
                    'miles': '12795179.575862522'},
  'orbiting_body': 'Venus',
  'relative_velocity': {'kilometers_per_hour': '53712.2832064585',
                        'kilometers_per_second': '14.9200786685',
                        'miles_per_hour': '33374.7239734797'}},
 {'close_approach_date': '1903-12-31',
  'close_approach_date_full': '1903-Dec-31 06:33',
  'epoch_date_close_approach': -2082907620000,
  'miss_distance': {'astronomical': '0.0509656055',
                    'kilometers': '7624346.026060285',
                    'lunar': '19.8256205395',
                    'miles': '4737548.937530533'},
  'orbiti

Notice that the `miss_distance` field contains the distance (in various units) by which the NEO missed an "orbiting body".

## Exercise 2

In the code cell below, write a python function that takes a list of "close approach data" as a parameter,
and returns a 2-tuple with the (date, miss km) of the closest approach to Earth in the list (where "miss km" is the miss distance in km).

Hints:
* notice the input is a list of dictionaries.  Each dictionary has a 'close_approach_date", "orbiting_body", and 'miss_distance' field.
* we are only interested in the closest approach to "Earth"
* use a loop if that is easier to understand - we will look at more compact algorithms to solve this problem in class.

Add at least one unit test to check your work - note the test data only needs dictionaries with the fields your function actually uses.


In [65]:
# Ex. 2 your code here
for x in close_approach_data:
  if x['orbiting_body'] == 'Earth':
   print(x['close_approach_date'], x['miss_distance']['kilometers'])

1903-12-31 7624346.026060285
1908-12-22 11057371.42805881
1913-12-13 12452611.356234633
1918-12-05 11576889.660109573
1923-11-29 10620153.446923031
1928-11-25 12905961.851992916
1933-01-04 27292943.215377312
1933-11-25 18715279.766173169
1938-01-06 18372620.324776766
1938-11-26 26471120.270472635
1943-01-06 10421756.976125836
1948-01-05 5444987.770215011
1953-01-02 5711630.205874513
1957-12-28 8824320.013345894
1962-12-21 11443290.444558001
1967-12-14 12335177.626676113
1972-12-05 11619189.594795885
1977-11-30 10615599.762559166
1982-11-27 12040210.210632515
1987-01-05 29317140.505230896
1987-11-26 17282564.165342368
1992-01-06 19591541.645247012
1992-11-26 25439262.649117521
1997-01-06 10892964.685390066
2002-01-05 5049126.658793064
2007-01-02 6257650.672981931
2011-12-26 9999774.071990431
2016-12-16 12156627.931297143
2021-12-08 11881875.765185367
2026-12-01 10693016.181570982
2031-11-27 12018582.62216981
2036-01-05 29172914.260908773
2036-11-25 17520129.248050183
2041-01-06 19584799

## Challenge - Take your skills to the next level...
## Exercise 3

In the code cell below, write a complete program that:
 1. fetches the list of NEO's for this week.
 2. for each NEO, fetch it's complete orbital data and determine its closest approach to Earth
 3. identify which NEO from this week's data makes the closet approach to earth
 4. print a nice message with information about the NEO, when it will approach the Earth, and how close it will come.

Hints:
* you'll need the start and end date - end date is today, see if you can use a [`timedelta`](https://docs.python.org/3/library/datetime.html#timedelta-objects) object to ge the start date (you can do basic "date math" with `timedelta` and `date` objects!)
* you may need to modify the function we wrote in Ex. 2 to return a triple with the NEO's id included;
* lots of opportunity here for more practice with list comprehensions


In [ ]:
# Ex. 3 (challenge) your code here
